In [ ]:
# Импорт необходимых модулей
import os
import pickle
from classes.audio_analyzer import AudioSceneAnalyzer
from classes.video_analyzer_complex import VideoPipeline
from classes.scene_enricher import SceneEnricher
from classes.openai import OpenAIThemer
from classes.simple_transcriptor import ASRProcessor


c:\Users\d.krasnov.kulturauao\Projects\My_projects\Hachakaton\kion_practice\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
VIDEO_PATH = "videos/Video_01.mp4"          # Путь к видеофайлу
AUDIO_PATH = "temp/temp_audio.wav"           # Временный WAV
OUTPUT_DIR = "output_files/"
TEMP_DIR = "temp/"         # Папка для результатов
CACHE_PATH = os.path.join(TEMP_DIR, "results.pkl")
MIN_SCENE_LENGTH = 2.0                  # Мин. длительность сцены
MAX_SCENE_LENGTH = 300.0                # Макс. допустимая длина сцены (5 мин)


# Анализ видео и выделение сцен

### В этой ячейке мы создаём объект VideoAnalyzer и анализируем видео для получения сцены и лиц, привязанных к трекам.


In [ ]:
if os.path.exists(CACHE_PATH):
    print("🔁 Загружаем из кеша...")
    with open(CACHE_PATH, "rb") as f:
        scenes, track_faces, tracking_frames, track_id_to_person = pickle.load(f)

else:
    pipeline = VideoPipeline()
    scenes, track_faces, tracking_frames, track_id_to_person = pipeline.run(VIDEO_PATH, OUTPUT_DIR)

    with open(CACHE_PATH, "wb") as f:
        pickle.dump((scenes, track_faces, tracking_frames, track_id_to_person), f)

# Анализ аудио: распознавание речи и активность

### Используем AudioAnalyzer для транскрибирования речи и измерения активности звука (энергии).

In [ ]:
splitter = AudioSceneAnalyzer(
    video_path=VIDEO_PATH,
    output_dir=OUTPUT_DIR,
    whisper_model_size='small'
)
splitter.run(sensitivity=0.88, min_scene_duration=2.0)
segments = splitter.transcribe_audio()
energy = splitter.detect_audio_activity(frame_duration=1.0)

# Обогащение сцен транскриптами и аудио энергией

### Добавляем к сценам распознанный текст и усреднённые показатели звуковой активности.


In [ ]:
enricher = SceneEnricher(segments, energy)
scenes_final = enricher.run(scenes, track_id_to_person)
#scenes_final = enricher.run(scene_data, track_id_to_person, print_report=False)


In [ ]:
asr = ASRProcessor()
segments_short = asr.process(VIDEO_PATH)
themer = OpenAIThemer(
        api_key="",  # Добавь свой ключ
        base_url="https://api.proxyapi.ru/openai/v1"
    )
themes = themer.get_themes(segments_short, audio_path=VIDEO_PATH)